In [3]:
from trustifai import Trustifai, MetricContext
from llama_index.core import Document
from IPython.display import display, Markdown

In [7]:
generate_engine = Trustifai(config_path="../config_file.yaml")

In [8]:
query = "What is Acme Corp's policy on remote work?"
answer = generate_engine.generate(prompt=query)

In [9]:
answer

{'response': "I don't have access to Acme Corp's specific internal policies. However, companies typically outline their remote work policies in employee handbooks or on their official website. These policies often cover eligibility, expectations for communication and productivity, equipment provisions, and security requirements.\n\nIf you are an employee or have access to Acme Corp's internal resources, I recommend checking the company's intranet, HR portal, or reaching out to your manager or HR representative for the most accurate and up-to-date information regarding their remote work policy. If you provide more context or details, I may be able to offer more tailored guidance.",
 'metadata': {'confidence_score': 0.67,
  'confidence_label': 'Low Confidence',
  'confidence_details': {'explanation': 'Model is uncertain about its output.',
   'avg_logprob': -0.29,
   'variance': 0.1,
   'token_count': 120},
  'logprobs_available': True}}

In [10]:
documents = [
    Document(
        page_content="Acme Corp announced a hybrid work model in 2023, requiring employees to be in office 3 days per week.",
        metadata={"source": "hr_policy_2023.pdf"}
    ),
    Document(
        page_content="The company provides home office stipends of up to $500 for remote setup.",
        metadata={"source": "benefits_guide.pdf"}
    )
]

metric_context = MetricContext(
    query=query,
    answer=answer['response'],
    documents=documents,
)
trust_engine = Trustifai(metric_context, "../config_file.yaml")

In [11]:
trust_score = trust_engine.get_trust_score()
trust_score

{'score': 0.48,
 'label': 'UNRELIABLE',
 'details': {'evidence_coverage': {'score': 0.0,
   'label': 'Likely Hallucinated Answer',
   'details': {'explanation': 'Many claims lack support from source documents.',
    'strategy': 'LLM',
    'total_sentences': 5,
    'supported_sentences': 0,
    'unsupported_sentences': ["I don't have access to Acme Corp's specific internal policies.",
     'However, companies typically outline their remote work policies in employee handbooks or on their official website.',
     'These policies often cover eligibility, expectations for communication and productivity, equipment provisions, and security requirements.',
     "If you are an employee or have access to Acme Corp's internal resources, I recommend checking the company's intranet, HR portal, or reaching out to your manager or HR representative for the most accurate and up-to-date information regarding their remote work policy.",
     'If you provide more context or details, I may be able to offer

In [12]:
reasoning_graph = trust_engine.build_reasoning_graph(trust_score)
mermaid_graph = trust_engine.visualize(reasoning_graph, graph_type='mermaid')
pyvis_graph = trust_engine.visualize(reasoning_graph, graph_type='pyvis') #exported to html file

In [13]:
display(Markdown(mermaid_graph))

```mermaid
flowchart TD
   evidence_coverage["<b>Evidence Coverage</b><br/>Score: 0.00<br/>Likely Hallucinated Answer"]
   semantic_drift["<b>Semantic Drift</b><br/>Score: 0.67<br/>Partial Alignment"]
   consistency["<b>Consistency</b><br/>Score: 0.96<br/>Stable Consistency"]
   source_diversity["<b>Source Diversity</b><br/>Score: 0.85<br/>High Trust"]
   trust_aggregation{"<b>Trust Score</b><br/>Score: 0.48"}
   final_decision("<b>Decision: UNRELIABLE</b>")
    evidence_coverage --> trust_aggregation
    semantic_drift --> trust_aggregation
    consistency --> trust_aggregation
    source_diversity --> trust_aggregation
    trust_aggregation --> final_decision
    style evidence_coverage fill:#ff6b6b,color:#000000
    style semantic_drift fill:#f39c12,color:#000000
    style consistency fill:#2ecc71,color:#000000
    style source_diversity fill:#2ecc71,color:#000000
    style trust_aggregation fill:#ff6b6b,color:#000000
    style final_decision fill:#ff6b6b,color:#000000
```